In [ ]:
# https://github.com/Newmu/Theano-Tutorials
# see batch normalization

In [1]:
import datasets

datasets.downloads = '/home/greg/Downloads/'

trX, teX, trY, teY = datasets.mnist()

In [129]:
import numpy as np

In [165]:
X = trX
Y = trY

labels = np.argmax(teY, axis = 1)

epochs = 1
H = 625

N, D = X.shape
N, C = Y.shape

In [ ]:
import tensorflow as tf

def layer(x, shape, activator):
    weights = lambda shape: tf.Variable(tf.random_normal_initializer(0.0, 0.05)(shape))
    zeros = lambda shape: tf.Variable(tf.zeros(shape, dtype = tf.float32))
    return activator(tf.matmul(x, weights(shape)) + zeros(shape[1]))  
    
def fit(X, Y, optimizer, epochs):
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        for _ in range(epochs):
            for start, end in zip(range(0, N - 32, 32), range(32, N, 32)):
                feed_dict = {x: X[start : end], y: Y[start : end]}
                sess.run(optimizer, feed_dict = feed_dict)
            p = model.eval({x: teX})
    
    c = np.argmax(p, axis = 1)
    print np.mean(c == labels) 
            

In [249]:
import tensorflow as tf

x = tf.placeholder('float')
y = tf.placeholder('float')

weights = lambda shape: tf.Variable(tf.random_normal_initializer(0.0, 0.05)(shape))
zeros = lambda shape: tf.Variable(tf.zeros(shape, dtype = tf.float32))
layer = lambda x, shape, activator: activator(tf.matmul(x, weights(shape)) + zeros(shape[1]))

l = layer(x, [D, H], tf.nn.relu)  
model = layer(l, [H, C], tf.nn.softmax)  
cost = tf.nn.softmax_cross_entropy_with_logits(model, y)
optimizer = tf.train.AdamOptimizer().minimize(cost)

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    for _ in range(epochs):
        for start, end in zip(range(0, N - 32, 32), range(32, N, 32)):
            feed_dict = {x: X[start : end], y: Y[start : end]}
            sess.run(optimizer, feed_dict = feed_dict)

    p = model.eval({x: teX})
    
c = np.argmax(p, axis = 1)
print np.mean(c == labels) 

0.9498


In [238]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(output_dim = H, input_dim = D, activation = 'relu'))
model.add(Dense(output_dim = C, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
model.fit(X, Y, nb_epoch = epochs)

p = model.predict_proba(teX, verbose = 0)
c = np.argmax(p, axis = 1)
print np.mean(c == labels)

Epoch 1/1
60000/60000 [==============================] - 24s - loss: 0.1951    
0.9681


In [186]:
config = model.get_config()
weights = model.get_weights()

model = Sequential.from_config(config)
model.set_weights(weights)